# Analyzing OpenRTB Bids

This notebook analyzes bid request/response pairs from OpenRTB data to extract placement IDs with their corresponding bid prices from each demand source. The following fields are commonly used to identify placement IDs in OpenRTB:

1. `imp.ext.gpid` - Global Placement ID
2. `imp.tagid` - Tag ID
3. `imp.ext.data.pbadslot` - Publisher Ad Slot
4. `imp.id` - Impression ID
5. `imp.ext.dfp_id` - DFP Ad Unit Code

We'll extract these placement identifiers along with the bid prices for each demand source.

In [1]:
try:
    import pandas as pd
except ImportError:
    import sys
    !{sys.executable} -m pip install pandas

In [2]:
import json
import os
import re
from glob import glob
import urllib.parse
from src.bid_analysis import extract_bids_from_responses, extract_placements_from_requests

# Find all bid data files in the bids directory
bid_files = glob("../data/bids/*.json")
print(f"Found {len(bid_files)} bid data files")

Found 3 bid data files


In [ ]:
# Process all bid data files
all_bid_data = []

for bid_file in bid_files:
    print(f"Processing bid data file: {bid_file}")
    
    # Load the bid data
    with open(bid_file, 'r', encoding='UTF-8') as f:
        bid_data = json.load(f)
    
    # Extract requests and responses
    requests = bid_data.get('requests', [])
    responses = bid_data.get('responses', [])
    
    print(f"Found {len(requests)} requests with placement information")
    print(f"Found {len(responses)} bid responses")
    
    # Extract placement IDs from requests
    placement_map = extract_placements_from_requests(requests)
    
    # Extract bids from responses
    bids = extract_bids_from_responses(responses)
    
    # Add placement IDs to bid data by matching request and impression IDs
    for bid in bids:
        req_id = bid['request_id']
        imp_id = bid['placement_id']  # This is actually the impid from the bid
        
        # Look up the actual placement ID from our mapping
        if req_id in placement_map and imp_id in placement_map[req_id]:
            bid['placement_id'] = placement_map[req_id][imp_id]
    
    all_bid_data.extend(bids)
    print(f"Extracted {len(bids)} bid records")

Processing bid data file: ../data/bids\openrtb-bid-data-2025-10-01T01-35-07.json
Found 18 requests with placement information
Found 18 bid responses


KeyError: 'impid'

In [4]:
import pandas as pd
# Create a DataFrame from the extracted bid data
df = pd.DataFrame(all_bid_data)

# If the DataFrame is empty, create a sample with the right structure
if df.empty:
    df = pd.DataFrame(columns=['request_id', 'placement_id', 'bid_price', 'bid_currency', 
                               'demand_source', 'advertiser_domain', 'creative_id', 
                               'creative_width', 'creative_height'])

In [5]:
print(f"Total bid records: {len(df)}")
print("\nSummary statistics for bid prices:")
df[['bid_price']].describe()

Total bid records: 137

Summary statistics for bid prices:


,bid_price
count,137.000000
mean,11.921239
std,23.539388
min,0.020200
25%,0.038500
50%,1.204000
75%,12.000000
max,79.000000


In [6]:
print("\nTop 10 highest bids:")
df.sort_values('bid_price', ascending=False).head(10)[['request_id', 'placement_id', 'bid_price', 'bid_currency', 'demand_source', 'advertiser_domain']]


Top 10 highest bids:


,request_id,placement_id,bid_price,bid_currency,demand_source,advertiser_domain
91,10972.10606,/5129/ndm.taus/home#ad-block-728x90-1,79.0,AUD,casalemedia.com,icmarkets.com
96,10972.11441,/5129/ndm.taus/home#ad-block-728x90-1,77.0,AUD,casalemedia.com,icmarkets.com
101,10972.12279,/5129/ndm.taus/home#ad-block-728x90-1,77.0,AUD,casalemedia.com,icmarkets.com
25,10972.532,/5129/ndm.taus/home#ad-block-728x90-1,76.0,AUD,casalemedia.com,icmarkets.com
106,10972.13123,/5129/ndm.taus/home#ad-block-728x90-1,76.0,AUD,casalemedia.com,icmarkets.com
111,10972.13968,/5129/ndm.taus/home#ad-block-728x90-1,76.0,AUD,casalemedia.com,icmarkets.com
68,10972.6426,/5129/ndm.taus/home#ad-block-728x90-1,74.0,AUD,casalemedia.com,icmarkets.com
39,10972.1389,/5129/ndm.taus/home#ad-block-728x90-1,74.0,AUD,casalemedia.com,icmarkets.com
116,10972.14806,/5129/ndm.taus/home#ad-block-728x90-1,74.0,AUD,casalemedia.com,icmarkets.com
49,10972.3069,/5129/ndm.taus/home#ad-block-728x90-1,73.0,AUD,casalemedia.com,icmarkets.com


In [7]:
print("\nTop 10 most frequent placement IDs:")
# Count occurrences of each placement ID
placement_counts = df['placement_id'].value_counts().reset_index()
placement_counts.columns = ['placement_id', 'frequency']
placement_counts.head(10)


Top 10 most frequent placement IDs:


,placement_id,frequency
0,3,20
1,4,20
2,1,20
3,2,20
4,/5129/ndm.taus/home#ad-block-728x90-1,16
5,/1001609/Discuss_Web_HOME_MR1,8
6,/1001609/Discuss_Web_HOME_BBLive,5
7,/1001609/Discuss_Web_HOME_BB1,5
8,/1001609/Discuss_Web_HOME_BBLive2,4
9,/1001609/Discuss_Web_HOME_BBLive3,4


In [12]:
df[df['placement_id'].isin(['1'])]

,request_id,placement_id,bid_price,bid_currency,demand_source,advertiser_domain,creative_id,creative_width,creative_height
8,13772.399,1,1.0498,USD,api.taboola.com,interactivebrokers.com.hk,2138837301207346617,970,250
35,9232.379,1,1.2075,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
40,9232.1090,1,1.2040,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
45,9232.1850,1,1.1406,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
50,9232.2530,1,1.1636,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
55,9232.3208,1,1.2485,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
59,9232.3926,1,1.2143,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
64,9232.4594,1,1.1884,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
69,9232.5349,1,1.1715,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250
74,9232.6061,1,1.2089,USD,api.taboola.com,interactivebrokers.com.hk,3449672606546812601,970,250


In [8]:
print("\nAverage bid price by demand source (top 10):")
# Group by demand source and calculate average bid price and count
demand_source_stats = df.groupby('demand_source').agg({
    'bid_price': 'mean',
    'placement_id': 'count'
}).reset_index()
demand_source_stats.columns = ['demand_source', 'avg_bid_price', 'bid_count']
demand_source_stats.sort_values('avg_bid_price', ascending=False).head(10)


Average bid price by demand source (top 10):


,demand_source,avg_bid_price,bid_count
1,casalemedia.com,42.162162,37
0,api.taboola.com,0.810080,82
3,pixel.quantserve.com,0.425335,1
4,sg2-bid.adsrvr.org,0.393799,16
2,ox-rtb-asia-east1.openx.net,0.057000,1


In [9]:
print("\nBid price distribution by placement ID (top 5 placements with highest average bid price):")
# Group by placement ID and calculate bid statistics
placement_stats = df.groupby('placement_id').agg({
    'bid_price': ['mean', 'min', 'max'],
    'request_id': 'count'
}).reset_index()

# Flatten column names
placement_stats.columns = ['placement_id', 'avg_bid_price', 'min_bid', 'max_bid', 'bid_count']
placement_stats.sort_values('avg_bid_price', ascending=False).head(5)


Bid price distribution by placement ID (top 5 placements with highest average bid price):


,placement_id,avg_bid_price,min_bid,max_bid,bid_count
14,/5129/ndm.taus/home#ad-block-728x90-1,73.937500,62.000000,79.0,16
4,/1001609/Discuss_Web_HOME_BBLive4,14.267512,0.070050,19.0,4
1,/1001609/Discuss_Web_HOME_BBLive,13.980108,0.433372,24.0,5
3,/1001609/Discuss_Web_HOME_BBLive3,13.767702,0.070808,19.0,4
2,/1001609/Discuss_Web_HOME_BBLive2,13.623975,0.495899,19.0,4
